Jupyter Notebook to accompany:
"Loop Flexibility Enables Remote Transduction of Energy into the Enolase Active Site" by Emily J. Thompson, Adhayana Paul, Anthony T. Iavarone, and Judith P. Klinman


Uploaded Sep 1, 2020

In [ ]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# modules

import os
import math
import numpy as np
import sympy as sym
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from scipy import stats
from imp import reload as rl
from glob import glob as glob
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from xml.etree import ElementTree as et


# settings
## pandas
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 100)
pd.options.mode.chained_assignment = None

## matplot
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
# choosing colors for the graphs
# colors9 = blue --> grey --> red

temps = ('10C', '20C', '25C', '30C', '40C')
print(temps)

colorset = ["#0c2c84", "#1d91c0", "#74a9cf", "#bababa", "#fb6a4a", "#ef3b2c", "#b10026"]
CS = sns.color_palette(colorset)
allblack = sns.color_palette(['black','black','black','black','black','black','black'])

# print colors to see:
sns.palplot(CS)
sns.palplot(allblack)

# temperature:color code for plotting:
temp2color = {temp:color for temp, color in zip(temps, CS)}
temp2colorblack = {temp:color for temp, color in zip(temps, colorblack)}

### Generating Pandas table from XML files

In [ ]:
#generate back exchange file first from HDX Workbench XML output

files = glob('XML_BE/*.xml')
#print(files)

BE = []

for i in files:
    tree = et.parse(i)
    root = tree.getroot()

    df = {'start':[], 'end':[], 'charge':[], 'sequence':[], 'temperature':[], 'timepoint':[], 'replicate':[], 'percentD':[], 'valid':[]}

    for peptide in root.iter('Peptide'):
        for sample in peptide.getchildren():
            for datapoint in sample.getchildren():
                for rep in datapoint.getchildren():
                    if len(peptide.attrib['start']) > 0:
                        df['start'].append(peptide.attrib['start'])
                    else:
                        df['start'].append(np.nan)
                    df['end'].append(float(peptide.attrib['end']))
                    df['charge'].append(float(peptide.attrib['charge']))
                    df['sequence'].append(peptide.attrib['sequence'])
                    df['timepoint'].append(float(datapoint.attrib['name'].strip().replace('s','')))
                    df['temperature'].append(sample.attrib['name'])
                    df['replicate'].append(rep.attrib['name'])
                    df['percentD'].append(float(rep.attrib['percentD']))
                    df['valid'].append(rep.attrib['valid'])
    BE.append(pd.DataFrame(df))

BE = pd.concat(BE)
BE['percentD_good'] = np.where(BE['valid']=='true', BE.percentD, np.NaN) 
BE['start'] = BE['start'].apply(float)
BE.sort_values(['start','end','charge','temperature','timepoint','replicate'],inplace=True)
BE.to_csv('XML_BE/BE.tsv', sep="\t")
BE

In [ ]:
# curate BE.tsv file to only contain designated peptide set.
# Further datasets will be reduced to only contain peptides in the BE_df 

BE_df = pd.DataFrame(pd.read_csv('XML_BE/BE_curated.tsv', sep='\t'))
BE_df['start'] = BE_df['start'].apply(float)
BE_df['charge'] = BE_df['charge'].apply(float)
BE_df['end'] = BE_df['end'].apply(float)
BE_df

In [ ]:
# Upload all temperature dependent HDX-MS data from XML files and tabulate into dataframe
# percentD_good reports on the HDX Workbench value that can be toggled to include or not include a data point

files = glob('XML_data/*.xml')

WT = []

for i in files:
    tree = et.parse(i)
    root = tree.getroot()

    df = {'start':[], 'end':[], 'charge':[], 'sequence':[], 'temperature':[], 'timepoint':[], 'replicate':[], 'percentD':[], 'valid':[]}

    for peptide in root.iter('Peptide'):
        for sample in peptide.getchildren():
            for datapoint in sample.getchildren():
                for rep in datapoint.getchildren():
                    if len(peptide.attrib['start']) > 0:
                        df['start'].append(peptide.attrib['start'])
                    else:
                        df['start'].append(np.nan)
                    df['end'].append(float(peptide.attrib['end']))
                    df['charge'].append(float(peptide.attrib['charge']))
                    df['sequence'].append(peptide.attrib['sequence'])
                    df['timepoint'].append(float(datapoint.attrib['name'].strip().replace('s','')))
                    df['temperature'].append(sample.attrib['name'])
                    df['replicate'].append(rep.attrib['name'])
                    df['percentD'].append(float(rep.attrib['percentD']))
                    df['valid'].append(rep.attrib['valid'])
    WT.append(pd.DataFrame(df))

WT = pd.concat(WT)
WT['percentD_good'] = np.where(WT['valid']=='true', WT.percentD, np.NaN) 
WT['start'] = WT['start'].apply(float)
WT

In [ ]:
# Merge BE and WT dataframes such that only peptides in the BE dataframe are used for further analysis
# converting time point data from seconds into minutes


merged = {}
merged = pd.merge(left=BE_df, right=WT, on=['start', 'end', 'charge'])
merged['percentD_corr']=merged['percentD_good']*100/merged['BE']
merged['timepoint']=merged['timepoint'].astype(np.int64)
merged['replicate']=merged['replicate'].astype(np.int64)
merged['timepoint'] = (merged['timepoint'])/60
merged['AAs']=merged['sequence'].str.len()
merged['#P']=merged['sequence'].str.count('P')
merged['NH-total']=merged['AAs']-merged['#P']-2
merged['Daltons']=merged['percentD_corr']/100*merged['NH-total']
merged.sort_values(['start','end','charge','temperature','timepoint','replicate'],inplace=True)

merged.to_csv('merged.tsv', sep="\t")
merged

In [ ]:
#calculate average and standard deviation of replicate data

merged2 ={}
merged2['start'] = []
merged2['end'] = []
merged2['charge'] = []
merged2['temperature'] = []
merged2['timepoint'] = []
merged2['avg'] = []
merged2['sd'] = []
    
esubs = ['charge', 'start', 'end', 'temperature', 'timepoint']
for e, ee in merged.groupby(by = esubs):
    avgv = ee['Daltons'].mean()
    stdv = ee['Daltons'].std()
    merged2['avg'].append(avgv)
    merged2['sd'].append(stdv)
    merged2['start'].append(ee['start'].iloc[0])
    merged2['end'].append(ee['end'].iloc[0])
    merged2['charge'].append(ee['charge'].iloc[0])
    merged2['temperature'].append(ee['temperature'].iloc[0])
    merged2['timepoint'].append(ee['timepoint'].iloc[0])
merged2 = pd.DataFrame(merged2)
merged2

mergedc = pd.merge(merged, merged2, how = 'outer', on = ['timepoint', 'charge', 'start', 'end', 'temperature'])
mergedc

In [ ]:
#make a new column for the max exchange value (NH_exchange_max) for each unique dataset - this value will be used in the fitting

merged3 = {}
merged3['NH_exchange_max'] = []
merged3['start'] = []
merged3['end'] = []
merged3['charge'] = []
merged3['temperature'] = []
substt = ['charge', 'start', 'end', 'temperature']
for add, adds in mergedc.groupby(by = substt):
    tt = adds['avg'].max()
    merged3['NH_exchange_max'].append(tt)
    merged3['start'].append(adds['start'].iloc[0])
    merged3['end'].append(adds['end'].iloc[0])
    merged3['charge'].append(adds['charge'].iloc[0])
    merged3['temperature'].append(adds['temperature'].iloc[0])
merged3 = pd.DataFrame(merged3)


WTmerged = pd.merge(mergedc, merged3, 
               how = 'outer', 
               on = ['charge', 'start', 'end', 'temperature'])

WTmerged.to_csv('WTmerged.tsv', sep="\t")
WTmerged

In [ ]:
# define equation for fitting (Exp3a) 
# define fitting and plotting parameters: plot errorbars based on SD, use plot(moreX...) to make a smooth line, return statement 'no fit' if data cannot be fit to the equation
# define bounds and initial values when plotting

def Exp3a(x, a, b, c, d, e, f, g):
    return g-a*(np.exp(-d*x))-b*(np.exp(-e*x))-c*(np.exp(-f*x))

def plotfit(df, x, y, yerr, hue, func, colors, p0, bounds):
    for category, subset in df.groupby(by = [hue]):
        X = subset[x].values
        Y = subset[y].values
        YR = subset[yerr].values
        try:
            popt, pcov = curve_fit(func, X, Y, p0=pinitial, bounds=bbounds)
            moreX = np.arange(int(min(X)), int(max(X)), 0.001)       
            plt.plot(moreX, func(moreX, *popt),
                     label = ('T = %s' % 
                              (tempInfo[2])), 
                     color = colors[category])
            plt.errorbar(X, Y, YR, fmt = 'none', ecolor='gray', elinewidth=1, capsize=3)
            #plt.legend(bbox_to_anchor=(1.1, 1), loc=2, borderaxespad=0.)
            sns.scatterplot(data = upepDF, 
                    x = 'timepoint', y = 'avg', hue = 'temperature', 
                    palette = temp2color, legend = False)
        except RuntimeError:
                print('no fit') 

In [ ]:
'''
Plot as Semi-log
fitting to 3 exponentials
''' 

groups2 = WTmerged.groupby(by = ['start', 'end', 'charge', 'sequence', 'NH-total'], as_index = False)
num_groups = len(groups2)

for upep, upepDF in tqdm(groups2, total = num_groups):
    upepDF = upepDF.dropna(subset = ['avg'])
    ylim = upepDF['NH-total'].iloc[0]

    for tempInfo, tempDF in upepDF.groupby(by = ['start', 'end', 'temperature']):
        start, end, temperature = tempInfo
        Nobs = tempDF['NH_exchange_max'].iloc[0]
        if Nobs == 0:
            Nobs = 0.1
        pinitial = Nobs*0.5, Nobs*0.5, Nobs*0.5, 5, 0.5, 0.01, Nobs
        bbounds = ([0,0,0,2.5,0.05,0,Nobs*0.9],[Nobs*1.1,Nobs*1.1,Nobs*1.1,1000,2.5,0.05,Nobs*1.1])
 
        plotfit(tempDF, x = 'timepoint', y = 'avg', yerr = 'sd', hue = 'temperature',
                func = Exp3a, colors = temp2color, 
                p0 = pinitial, 
                bounds = bbounds)  

    plt.xlabel('Time (min)')
    plt.ylabel('Daltons')
    plt.xscale('log')
    plt.ylim(0, ylim)
    plt.xlim(0.1, 250)
    
#    sns.despine()
    sns.set_style("white")
    sns.set_style("ticks")
    sns.set_context("notebook", font_scale=1.3, rc={"lines.linewidth": 2})
    figname = 'SemiLog/%s-%s_c+%s.pdf' % (int(upep[0]), int(upep[1]), int(upep[2]))
    plt.title('%s-%s_c+%s, %s amides' % (int(upep[0]), int(upep[1]), int(upep[2]), ylim))
#    print(figname)
    plt.savefig(figname, bbox_inches='tight')
    plt.show();

    
# plt title gives peptide, charge, and number of reporting amides:
# eg 2-23_c+3, 19 amides = peptide 2-23, +3 charge, with 19 amides



In [ ]:
'''
plot with x-axis linear
'''        

groups2 = WTmerged.groupby(by = ['start', 'end', 'charge', 'sequence', 'NH-total'], as_index = False)
num_groups = len(groups2)

for upep, upepDF in tqdm(groups2, total = num_groups):
    upepDF = upepDF.dropna(subset = ['avg'])
    ylim = upepDF['NH-total'].iloc[0]

    for tempInfo, tempDF in upepDF.groupby(by = ['start', 'end', 'temperature']):
        start, end, temperature = tempInfo
        Nobs = tempDF['NH_exchange_max'].iloc[0]
        if Nobs == 0:
            Nobs = 0.1
        pinitial = Nobs*0.5, Nobs*0.5, Nobs*0.5, 5, 0.5, 0.01, Nobs
        bbounds = ([0,0,0,2.5,0.05,0,Nobs*0.9],[Nobs*1.1,Nobs*1.1,Nobs*1.1,1000,2.5,0.05,Nobs*1.1])

        plotfit(tempDF, x = 'timepoint', y = 'avg', yerr = 'sd', hue = 'temperature',
                func = Exp3a, colors = temp2color, 
                p0 = pinitial, 
                bounds = bbounds)  

    plt.xlabel('Time (min)', fontsize=20)
    plt.ylabel('Daltons', fontsize=20)
    plt.xscale('linear')
    plt.yscale('linear')
    plt.ylim(0, ylim)
    plt.xlim(-5, 250)
    
#    sns.despine()
    sns.set_style("white")
    sns.set_style("ticks")
    sns.set_context("notebook", font_scale=1.3, rc={"lines.linewidth": 2})
    figname = 'linear/%s-%s_c+%s.pdf' % (int(upep[0]), int(upep[1]), int(upep[2]))
    plt.title('%s-%s_c+%s, %s amides' % (int(upep[0]), int(upep[1]), int(upep[2]), ylim))
    print(figname)
#    plt.savefig(figname, bbox_inches='tight')
    plt.show();



In [ ]:
# defining how to put all the fitted data in a table

def fitdata(WTmerged, x, y, hue, func, colors, p0, bounds):
    k2_df = []
    k_df = {'variable':[], 'value':[], 'temperature':[], 'std.dev.res':[], 'DOF':[], 'Nobs':[]}
    for category, subset in WTmerged.groupby(by = ['temperature']):
        X = subset[x].values
        Y = subset[y].values
        try: 
            Nobs = subset['NH_exchange_max'].iloc[0]
            if Nobs == 0:
                Nobs = 0.1
            pinitial = Nobs*0.5, Nobs*0.5, Nobs*0.5, 10, 0.5, 0.01, Nobs
            bbounds = ([0,0,0,2.5,0.05,0,Nobs*0.9],[Nobs*1.1,Nobs*1.1,Nobs*1.1,1000,2.5,0.05,Nobs*1.1])
            
            popt, pcov = curve_fit(func, X, Y, p0 = pinitial, bounds = bbounds)
            SDR = ((sum((Y - func(X, *popt))**2))/(len(Y)-len(p0)-2))**0.5

            for variable, value in zip(['a', 'b', 'c', 'd', 'e', 'f', 'g'], popt):
                k_df['variable'].append(variable)
                k_df['value'].append(value)
                k_df['temperature'].append(category)
                k_df['std.dev.res'].append(SDR)
                k_df['DOF'].append(len(Y)-len(p0)-1)
                k_df['Nobs'].append(Nobs)

        except RuntimeError:
            for variable, value in zip(['a', 'b', 'c', 'd', 'e', 'f', 'g'], [np.nan]):
                k_df['variable'].append(variable)
                k_df['value'].append(value)
                k_df['temperature'].append(category)
                k_df['std.dev.res'].append(value)
                k_df['DOF'].append(len(Y)-len(p0)-1)
                k_df['Nobs'].append(Nobs)
        k2_df.append(pd.DataFrame(k_df))
    return pd.DataFrame(k_df)                                



In [ ]:
# putting the data in data table!
                    
  
WTresults = []
groups = WTmerged.groupby(by = ['sequence', 'charge', 'start', 'end'], as_index = False)
num_groups = len(groups)
for upep, upepDF in tqdm(groups, total = num_groups):
    upepDF = upepDF.dropna(subset = ['Daltons'])  

    results = fitdata(upepDF, x = 'timepoint', y = 'Daltons', hue = 'temperature',
                          func = Exp3a, colors = temp2color, p0 = pinitial, bounds = bbounds)
    results['sequence'] = [upep[0] for i in results['variable']]
    results['charge'] = [upep[1] for i in results['variable']]
    results['start'] = [upep[2] for i in results['variable']]
    results['end'] = [upep[3] for i in results['variable']]
    WTresults.append(results) 

WTresults = pd.concat(WTresults)
WTresults.to_csv('WT_results.tsv', sep="\t")
WTresults


In [ ]:
# extracting the variable:value data and putting it into named individual columns

subs = ['sequence', 'charge', 'start', 'end', 'temperature', 'std.dev.res']
WTresults2 = {i:[] for i in subs}
WTresults2['k1'] = []
WTresults2['k2'] = []
WTresults2['k3'] = []
WTresults2['k1_NH'] = []
WTresults2['k2_NH'] = []
WTresults2['k3_NH'] = []
WTresults2['Nobs_calc'] = []

for pept, ResultsDF in WTresults.groupby(by = subs):
    # store values to new dataframe
    for sub, value in zip(subs, pept):
        WTresults2[sub].append(value)
    v = {variable:value for variable, value in zip(ResultsDF['variable'], ResultsDF['value'])}
    WTresults2['k1'].append(v['d'])
    WTresults2['k2'].append(v['e'])
    WTresults2['k3'].append(v['f'])
    WTresults2['k1_NH'].append(v['a'])
    WTresults2['k2_NH'].append(v['b'])
    WTresults2['k3_NH'].append(v['c'])    
    WTresults2['Nobs_calc'].append(v['g'])  
    
WTresults2 = pd.DataFrame(WTresults2)
WTresults2

# combine merged and results tables

WTresults3 = pd.merge(WTresults2, WTmerged, 
               how = 'outer', 
               on = ['sequence', 'charge', 'start', 'end', 'temperature',])
WTresults3.sort_values(by=['start', 'end', 'charge', 'temperature', 'timepoint'])

WT_minimal = WTresults3.drop(columns = ['percentD', 'timepoint','valid','percentD_good', 'percentD_corr', 'BE', 'replicate',])
WT_minimal = WT_minimal.drop_duplicates()
WT_minimal = WT_minimal.sort_values(['start','end','charge','temperature'])
WT_minimal.to_csv('WT_minimal.tsv', sep="\t")
WT_minimal







In [ ]:
 '''modify WT_minimal --> 
    strip C from temperature and make it an integer
    make new column for 1/T(K)
    make new column for ln()
    math.log(x) = natural logarithm of x
'''   

def func_invT(x):
    return 1000/(273.15+x)
def func_lnk(x):
    try:
        return math.log(x)
    except:
        return np.nan

WT_minimal2 = WT_minimal.drop(columns = ['std.dev.res', 'Daltons', 'sd', 'avg'])
WT_minimal2['temp. degrees'] = [int(i.replace('C','')) for i in WT_minimal2['temperature']]
WT_minimal2['1000/T (K)'] = func_invT(WT_minimal2['temp. degrees'])
WT_minimal2['ln(p2*k2+p3*k3)_NHtotal'] = [func_lnk(i) for i in ((WT_minimal2['k2']*WT_minimal2['k2_NH']+WT_minimal2['k3']*WT_minimal2['k3_NH'])/WT_minimal2['NH-total'])]
WT_minimal2 = WT_minimal2.drop_duplicates()
WT_minimal2.to_csv('WT_minimal2.tsv', sep="\t")
WT_minimal2

In [ ]:
# defining linear regression calculation for Ea(HDX) and plotting
# Plot each graph and put Ea and SE values in legend

def linfit(df, x, y, color):
    
    Ea_df = {'slope':[], 'Ea':[], 'r^2':[], 'SE':[]}
    X = df[x].values
    Y = df[y].values
    slope, intercept, r_value, p_value, std_err = stats.linregress(X,Y)
    R = 1.987
    Ea = float(-slope*R)
    r2 = float(r_value**2)
    SE_R = float(std_err*R)
    plt.plot(X, intercept + slope*X, 'r', label=('Ea = %2.1f, SE = %2.1f' % (Ea, SE_R)), color = color )
    plt.legend(bbox_to_anchor=(1, 1), loc=2, borderaxespad=0.)
    plt.xlim(3.13,3.6)
    plt.xlabel('1000/T (1/K)')
    plt.ylabel('ln(kHDX)')
    Ea_df['slope'].append(slope)
    Ea_df['Ea'].append(Ea)
    Ea_df['r^2'].append(r2)
    Ea_df['SE'].append(SE_R)
        
    return pd.DataFrame(Ea_df)

In [ ]:
# Plotting

for set1, setDF in WT_minimal2.groupby(by = ['start','end','charge']):
    start, end, charge = set1
    plt.figure(figsize=(3,3))
    sns.scatterplot(data = setDF, x='1000/T (K)',y='ln(p2*k2+p3*k3)_NHtotal', hue='temperature', palette = temp2colorblack, legend = False, label = 'ln(p2*k2+p3*k3)')
    Ea1_temp = linfit(setDF, x = '1000/T (K)', y = 'ln(p2*k2+p3*k3)_NHtotal', color = 'black')
    sns.set_style("ticks")    
    figname1 = 'WT_Ea/%s-%s_c+%s_n_Ea.pdf' % (int(set1[0]), int(set1[1]), int(set1[2]))
    plt.title('%s-%s_c+%s, %s amides' % (int(set1[0]), int(set1[1]), int(set1[2]), ylim))
    plt.savefig(figname1, bbox_inches='tight')
    plt.show();